# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from python.sql_queries import *

As running local docker contaitner, configured the connection as below to adjust.

In [3]:
#connect_setting="host=127.0.0.1 dbname=sparkifydb user=student password=student"
connect_setting="host=172.17.0.2 dbname=sparkifydb user=student password=student"

In [4]:
conn = psycopg2.connect(connect_setting)
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data')

In [7]:
filepath = song_files[0]
filepath

'/home/hk/git/DEND/project1/data/song_data/A/A/A/TRAAABD128F429CF47.json'

In [8]:
df = pd.read_json(filepath, lines=True, orient='columns')
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969


In [9]:
df.keys()

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [10]:
song_data = (df.values[0][6], df.values[0][7], df.values[0][1], df.values[0][9], df.values[0][8])
song_data

('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546)

Think below is better as the order of keys is not controllable always.

In [11]:
song_data_bykey = (df['song_id'][0], df['title'][0], df['artist_id'][0], df['year'][0], df['duration'][0])
song_data_bykey

('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
artist_data = (df.values[0][1], df.values[0][5], df.values[0][4], df.values[0][2], df.values[0][3])
artist_data

('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892)

In [14]:
artist_data_bykey = (df['artist_id'][0], df['artist_name'][0], df['artist_location'][0], df['artist_latitude'][0], df['artist_longitude'][0])
artist_data_bykey

('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files  = get_files("./data/log_data")

In [17]:
filepath = log_files[0]

In [18]:
df = pd.read_json(filepath, lines=True, orient='columns')
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Great Lake Swimmers,Logged In,Kevin,M,0,Arellano,215.11791,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Your Rocky Spine,200,1542931645796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Soziedad Alkoholika,Logged In,Kevin,M,1,Arellano,204.74730,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Va Bien,200,1542931860796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,Franz Ferdinand,Logged In,Kevin,M,2,Arellano,172.01587,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Eleanor Put Your Boots On,200,1542932064796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,Modest Mouse,Logged In,Kevin,M,3,Arellano,209.52771,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Float On,200,1542932236796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
4,Adam Lambert,Logged In,Kevin,M,4,Arellano,266.44853,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Aftermath,200,1542932445796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df = df[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Great Lake Swimmers,Logged In,Kevin,M,0,Arellano,215.11791,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Your Rocky Spine,200,1542931645796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Soziedad Alkoholika,Logged In,Kevin,M,1,Arellano,204.74730,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Va Bien,200,1542931860796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,Franz Ferdinand,Logged In,Kevin,M,2,Arellano,172.01587,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Eleanor Put Your Boots On,200,1542932064796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,Modest Mouse,Logged In,Kevin,M,3,Arellano,209.52771,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Float On,200,1542932236796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
4,Adam Lambert,Logged In,Kevin,M,4,Arellano,266.44853,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Aftermath,200,1542932445796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


In [20]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-23 00:07:25.796
1   2018-11-23 00:11:00.796
2   2018-11-23 00:14:24.796
3   2018-11-23 00:17:16.796
4   2018-11-23 00:20:45.796
Name: ts, dtype: datetime64[ns]

In [21]:
time_data = list(zip(t.dt.strftime('%Y-%m-%d %I:%M:%S'), t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday))
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [22]:
time_df = pd.DataFrame(time_data, columns=column_labels)
for i, row in time_df.iterrows():
    print(list(row))

['2018-11-23 12:07:25', 0, 23, 47, 11, 2018, 4]
['2018-11-23 12:11:00', 0, 23, 47, 11, 2018, 4]
['2018-11-23 12:14:24', 0, 23, 47, 11, 2018, 4]
['2018-11-23 12:17:16', 0, 23, 47, 11, 2018, 4]
['2018-11-23 12:20:45', 0, 23, 47, 11, 2018, 4]
['2018-11-23 12:25:11', 0, 23, 47, 11, 2018, 4]
['2018-11-23 12:29:30', 0, 23, 47, 11, 2018, 4]
['2018-11-23 12:33:55', 0, 23, 47, 11, 2018, 4]
['2018-11-23 12:38:05', 0, 23, 47, 11, 2018, 4]
['2018-11-23 12:42:32', 0, 23, 47, 11, 2018, 4]
['2018-11-23 12:45:07', 0, 23, 47, 11, 2018, 4]
['2018-11-23 12:49:02', 0, 23, 47, 11, 2018, 4]
['2018-11-23 01:34:56', 1, 23, 47, 11, 2018, 4]
['2018-11-23 01:39:24', 1, 23, 47, 11, 2018, 4]
['2018-11-23 02:35:36', 2, 23, 47, 11, 2018, 4]
['2018-11-23 02:39:36', 2, 23, 47, 11, 2018, 4]
['2018-11-23 03:48:24', 3, 23, 47, 11, 2018, 4]
['2018-11-23 03:52:36', 3, 23, 47, 11, 2018, 4]
['2018-11-23 03:57:58', 3, 23, 47, 11, 2018, 4]
['2018-11-23 04:01:13', 4, 23, 47, 11, 2018, 4]
['2018-11-23 04:04:51', 4, 23, 47, 11, 2

In [23]:
time_df.head(15)

,start_time,hour,day,week,month,year,weekday
0,2018-11-23 12:07:25,0,23,47,11,2018,4
1,2018-11-23 12:11:00,0,23,47,11,2018,4
2,2018-11-23 12:14:24,0,23,47,11,2018,4
3,2018-11-23 12:17:16,0,23,47,11,2018,4
4,2018-11-23 12:20:45,0,23,47,11,2018,4
5,2018-11-23 12:25:11,0,23,47,11,2018,4
6,2018-11-23 12:29:30,0,23,47,11,2018,4
7,2018-11-23 12:33:55,0,23,47,11,2018,4
8,2018-11-23 12:38:05,0,23,47,11,2018,4
9,2018-11-23 12:42:32,0,23,47,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [24]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run test.ipynb to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
df.keys()

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [26]:
user_df = df.loc[:,['userId','firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,66,Kevin,Arellano,M,free
1,66,Kevin,Arellano,M,free
2,66,Kevin,Arellano,M,free
3,66,Kevin,Arellano,M,free
4,66,Kevin,Arellano,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [27]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [28]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    start_time = pd.to_datetime(row.ts, unit='ms').strftime('%Y-%m-%d %I:%M:%S')
    songplay_data = (start_time, row.userId, row.level, str(songid), str(artistid), row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [29]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.